In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
rows_interval = 8
rows_min = rows_interval
rows_max = 256
rows = int(rows_max/rows_interval) - int(rows_min/rows_interval) + 1
row_space = list(range(rows_min, rows_max + 1, rows_interval))

columns_interval = rows_interval
columns_min = rows_min
columns_max = rows_max
columns = int(columns_max/columns_interval) - int(columns_min/columns_interval) + 1
column_space = list(range(columns_min, columns_max + 1, columns_interval))

batch_count = 1000

ns_conversion_factor = 1e-6 # ns to ms

In [ ]:
def array_operation(median_array_op_prec, op, prec):
    for i, n in enumerate(row_space):
        for j, m in enumerate(column_space):
            try:
                directoryname = f"../blas_results/gemvstridedbatched_{op}_{prec}/{n}_{m}_{batch_count}"
                filename = "results.csv"
                df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
                subtracted_df = df["EndNs"] - df["BeginNs"]
                subtracted_array = subtracted_df.to_numpy()
                median_array_op_prec[i, j] = np.median(subtracted_array) * ns_conversion_factor
            except:
                print(n,m)

In [ ]:
median_arrays = []
for precision in [32, 64]:
    for operation in ["op_n", "op_t"]:
        median_arrays.append(np.zeros((rows, columns)))
        array_operation(median_arrays[-1], operation, precision)

In [ ]:
for median_array, operation, precision in zip(median_arrays, ["op_n", "op_t", "op_n", "op_t"], [32, 32, 64, 64]):    
    fig, ax = plt.subplots(dpi=256)

    c = ax.imshow(median_array, 
                origin="lower", 
                interpolation='none',
                extent=[columns_min-columns_interval/2,
                        columns_max+columns_interval/2,
                        rows_min-rows_interval/2,
                        rows_max+rows_interval/2,
                        ],
    )
    plt.title(
        f"median time to perform gemvstridedbatched\n({operation}, {precision}-bit, batchCount={batch_count})"
    )
    plt.xlabel("Number of columns")
    plt.ylabel("Number of rows")
    cbar = plt.colorbar(c)

    cbar.ax.set_ylabel("milliseconds")
    plt.xticks(range(2*columns_min, columns_max + 1, 2*columns_interval))
    plt.yticks(range(2*rows_min, rows_max + 1, 2*rows_interval))

    plt.show()

In [ ]:
# op_n - op_t
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import BoundaryNorm
fig, ax = plt.subplots(dpi=256)

difference_array = median_arrays[2] - median_arrays[3]

c = ax.imshow(difference_array[0:8,0:8], 
              origin="lower", 
              interpolation='none',
              extent=[
                  columns_min-columns_interval/2,
                  columns_max+columns_interval/2,
                  rows_min-rows_interval/2,
                  rows_max+rows_interval/2,
              ],
              cmap="RdBu",
              vmin=-10,
              vmax=10,
)
plt.title(
    f"difference in median time to perform gemvstridedbatched\n(op_n - op_t, {precision}-bit, batchCount={batch_count})"
)
plt.xlabel("Number of columns")
plt.ylabel("Number of rows")

#colormap stuff
cbar = plt.colorbar(c)

cbar.ax.set_ylabel("milliseconds")
plt.xticks(range(2*columns_min, columns_max + 1, 2*columns_interval))
plt.yticks(range(2*rows_min, rows_max + 1, 2*rows_interval))

plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig2, axes = plt.subplots(
    2,
    2, 
    subplot_kw=dict(box_aspect=1), 
    layout="constrained",
    dpi=512,
)

fig.add_subplot(111, frameon=False)

titles = ["op_n_32", "op_n_64", "op_t_32", "op_t_64"]

for i, ax in enumerate(axes.flat):
    im = ax.imshow(median_arrays[i],
              origin="lower", 
              interpolation='none',
              extent=[columns_min-columns_interval/2,
                      columns_max+columns_interval/2,
                      rows_min-rows_interval/2,
                      rows_max+rows_interval/2,
                      ],
              )
    ax.title.set_text(titles[i])

    # color bars
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')
    cbar.ax.set_ylabel("milliseconds")

    ticks_stepsize = 4
    ax.set_xticks(range(2*ticks_stepsize*columns_min, columns_max + 1, 2*ticks_stepsize*columns_interval))
    ax.set_yticks(range(ticks_stepsize*rows_min, rows_max + 1, ticks_stepsize*rows_interval))

plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Number of columns")
plt.ylabel("Number of rows")

plt.show()